In [4]:
using Gen
using PyCall
using ProgressBars
pushfirst!(PyVector(pyimport("sys")."path"), "");


┌ Info: Precompiling ProgressBars [49802e3a-d2f1-5c88-81d8-b72133a6f568]
└ @ Base loading.jl:1260


PyObject ['', 'C:\\Users\\AJShah\\.julia\\conda\\3\\python37.zip', 'C:\\Users\\AJShah\\.julia\\conda\\3\\DLLs', 'C:\\Users\\AJShah\\.julia\\conda\\3\\lib', 'C:\\Users\\AJShah\\AppData\\Local\\Programs\\Julia\\Julia-1.4.0\\bin', 'C:\\Users\\AJShah\\.julia\\conda\\3', 'C:\\Users\\AJShah\\.julia\\conda\\3\\lib\\site-packages', 'C:\\Users\\AJShah\\.julia\\conda\\3\\lib\\site-packages\\win32', 'C:\\Users\\AJShah\\.julia\\conda\\3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\AJShah\\.julia\\conda\\3\\lib\\site-packages\\Pythonwin']

In [8]:
pyimport("sir")

PyCall.PyError: PyError (PyImport_ImportModule

The Python package sir could not be found by pyimport. Usually this means
that you did not install sir in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the sir module, you can
use `pyimport_conda("sir", PKG)`, where PKG is the Anaconda
package the contains the module sir, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run Pkg.build("PyCall"), and re-launch Julia.

) <class 'ModuleNotFoundError'>
ModuleNotFoundError("No module named 'scipy'")
 

In [9]:
ENV["PYTHON"]


KeyError: KeyError: key "PYTHON" not found